In [1]:
from bs4 import BeautifulSoup
import requests
import os
import wget
import time
base_path = "C:\\Users\\10235555\\Documents\\Dataportal Data\\Global map"
import urllib.request
import logging
from selenium.webdriver.support.ui import Select
import datetime
from Hashing.HashScrapedData import _hashing
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from functools import reduce
import time
import os
from RequestInferSchemaToJsonAPI.main import TriggerInferShemaToJsonAPI

In [5]:
chrome_options = webdriver.ChromeOptions()
base_path = "C:\\Users\\10235555\\Documents\\Dataportal Data\\Global map" # local, gets current working directory
prefs = {'download.default_directory' : base_path}#, "profile.content_settings.exceptions.automatic_downloads.*.setting" : 1}
chrome_options.add_experimental_option('prefs', prefs)
# chrome_options.add_argument('--user-agent="Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 640 XL LTE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10166"')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.maximize_window()

In [3]:
driver.get('https://di.unfccc.int/global_map')

In [4]:
iframe = driver.find_element(By.CLASS_NAME,'global-map')
driver.switch_to.frame(iframe)
content = driver.find_element(By.CLASS_NAME,'contentContainer')
title = driver.find_element(By.ID,'table_header_row').find_element(By.ID,'left_country_list_header').text.replace('\n',' ')
col_1 = title.split('- ')[1].split(',')[0]
# driver.switch_to.default_content()

In [6]:
from bs4 import BeautifulSoup
import requests
import os
import wget
import time
base_path = "\\\\10.30.31.77\\data_collection_dump\\RawData\\UNFCCC\\Global map"
import urllib.request
import logging
from selenium.webdriver.support.ui import Select
import datetime
from Hashing.HashScrapedData import _hashing
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from functools import reduce
import time
import os
from RequestInferSchemaToJsonAPI.main import TriggerInferShemaToJsonAPI

chrome_options = webdriver.ChromeOptions()
base_path = "\\\\10.30.31.77\\data_collection_dump\\RawData\\UNFCCC\\Global map" # local, gets current working directory
prefs = {'download.default_directory' : base_path}#, "profile.content_settings.exceptions.automatic_downloads.*.setting" : 1}
chrome_options.add_experimental_option('prefs', prefs)
# chrome_options.add_argument('--user-agent="Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 640 XL LTE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10166"')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.maximize_window()

driver.get('https://di.unfccc.int/global_map')
time.sleep(3)
notes = driver.find_element(By.CLASS_NAME,'disclaimer-notes').text
iframe = driver.find_element(By.CLASS_NAME,'global-map')
driver.switch_to.frame(iframe)
content = driver.find_element(By.CLASS_NAME,'contentContainer')
# title = driver.find_element(By.ID,'table_header_row').find_element(By.ID,'left_country_list_header').text.replace('\n',' ')
# col_1 = title.split('- ')[1].split(',')[0]
select_elementSector = driver.find_element(By.ID,'sector')
select_sector = Select(select_elementSector)
option_list_sector = select_sector.options
select_elementGas = driver.find_element(By.ID,'gas')
select_gas = Select(select_elementGas)
option_list_gas = select_gas.options
select_elementYear = driver.find_element(By.ID,'year')
select_year = Select(select_elementYear)
option_list_year = select_year.options
for op_s in option_list_sector[-1:]:
    select_sector.select_by_visible_text(op_s.text)
    t_s = op_s.text
    time.sleep(2)
    for op_g in option_list_gas:
        select_gas.select_by_visible_text(op_g.text)
        t_g = op_g.text
        time.sleep(2)
        title = 'All Annex I Parties' + ' - ' + t_s + ', ' + t_g + ', in ' + 'kt CO2 eq.'
        title = title.replace('.','')
        file = title + '.xlsx'
        # if file in os.listdir(base_path):
        title = title + ' 2'
        pdList = []
        for op_y in option_list_year:
            select_year.select_by_visible_text(op_y.text)
            t_y = op_y.text
            col_1 = t_y
            try:
                col_1 = col_1.split('_')[0]
            except:
                pass
            time.sleep(2)
            table = content.find_element(By.ID,'country_list_table')
            rows = table.find_elements(By.TAG_NAME,'tr')
            l = []
            for tr in rows:
                cells = tr.find_elements(By.TAG_NAME,'td')
                cells_text = [cell.text for cell in cells]
                l.append(cells_text)
            df_T = pd.DataFrame(l) 
            df_1 = df_T.iloc[:,1:3]
            df_1.columns = ['country',col_1]
            df_2 = df_T.iloc[:,5:7]
            df_2.columns = ['country',col_1]
            df_3 = df_T.iloc[:,9:11]
            df_3.columns = ['country',col_1]
            df_eachYear = pd.concat([df_1,df_2,df_3])
            pdList.append(df_eachYear)
        df = reduce(lambda  left,right: pd.merge(left,right,on=['country'],how='outer'), pdList) 
        df.to_excel(f'{base_path}\\{title}.xlsx',index=False)
        try:
            BodyDict = {
            "JobPath":f'//10.30.31.77/data_collection_dump/RawData/UNFCCC/Global map/{title}.xlsx', #* Point to downloaded data for conversion #
            "JsonDetails":{
                    ## Required
                    "organisation": "un-agencies",
                    "source": "UNFCCC",
                    "source_description" : "The United Nations Framework Convention on Climate Change established an international environmental treaty to combat 'dangerous human interference with the climate system', in part by stabilizing greenhouse gas concentrations in the atmosphere.",
                    "source_url" : "https://di.unfccc.int",
                    "table" : title ,
                    "description" : notes  ,
                    ## Optional
                    "JobType": "JSON",
                    "CleanPush": True,
                    "Server": "str",
                    "UseJsonFormatForSQL":  False,
                    "CleanReplace":True,
                    "MergeSchema": False,
                    "tags": [
                                {"name": 'Global map'}
                            ],
                    "additional_data_sources": [{       
                            "name": '',        
                            "url": ''  ## this object will be ignored if "name" is empty    }
                    }],
                    "limitations":'',
                    "concept":  '',
                    "periodicity":  '',
                    "topic": title ,
                    "created": '',                       #* this should follow the following formats %Y-%m-%dT%H:%M:%S" or "%Y-%m-%d"
                    "last_modified":'' ,                #* ""               ""                  ""              ""
                    "TriggerTalend" :  False,    #* initialise to True for production
                    "SavePathForJsonOutput": "//10.30.31.77/data_collection_dump/TestData/UNFCCC/Global map" #* initialise as empty string for production.
                }
            }
            # tablenom = BodyDict['JsonDetails']['table']

            # hashmessage = _hashing(BodyDict['JsonDetails']['source'], tablenom, BodyDict["JobPath"])

            # if hashmessage["Trigger_InferSchema"] == True and hashmessage["Success"] == True:

            TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
            TriggerInferShemaToJsonAPIClass.TriggerAPI()
            logging.info(f"Conversion successful - {title} ")
            print(BodyDict)
            #     logging.info(f"Conversion successful - {tablenom}, hashmessage: {hashmessage['message']}")

            # # logging.info(f"Conversion successful for {dataset}")
            # elif hashmessage['Success'] == True and hashmessage['Trigger_InferSchema'] == False:

            #     # dont trigger conversion nor talend
            #     logging.info(f"{hashmessage['message']}")

            # elif hashmessage['Success'] == False:
            #     logging.info(f"Hashing error or Unexpected Issue: {hashmessage['message']}")
        except  Exception as err:
            print(err)

{'JobPath': '//10.30.31.77/data_collection_dump/RawData/UNFCCC/Global map/All Annex I Parties - Other, Aggregate GHGs, in kt CO2 eq 2.xlsx', 'JsonDetails': {'organisation': 'un-agencies', 'source': 'UNFCCC', 'source_description': "The United Nations Framework Convention on Climate Change established an international environmental treaty to combat 'dangerous human interference with the climate system', in part by stabilizing greenhouse gas concentrations in the atmosphere.", 'source_url': 'https://di.unfccc.int', 'table': 'All Annex I Parties - Other, Aggregate GHGs, in kt CO2 eq 2', 'description': 'Note 1: The reporting and review requirements for GHG inventories are different for Annex I and non-Annex I Parties. The definition format of data for emissions/removals from the forestry sector is different for Annex I and non-Annex I Parties.\nNote 2: Base year data in the data interface relate to the base year under the Climate Change Convention (UNFCCC). The base year under the Conventio

In [19]:
table = content.find_element(By.ID,'country_list_table')
rows = table.find_elements(By.TAG_NAME,'tr')
l = []
for tr in rows:
    cells = tr.find_elements(By.TAG_NAME,'td')
    cells_text = [cell.text for cell in cells]
    l.append(cells_text)
df = pd.DataFrame(l) 
df_1 = df.iloc[:,1:3]
df_1.columns = ['country',col_1]
df_2 = df.iloc[:,5:7]
df_2.columns = ['country',col_1]
df_3 = df.iloc[:,9:11]
df_3.columns = ['country',col_1]
df = pd.concat([df_1,df_2,df_3])
df

,country,Total emissions with LULUCF Aggregate GHGs
0,Sweden,-81.29%
1,Romania,-72.90%
2,Ukraine,-65.32%
3,Estonia,-65.28%
4,Lithuania,-65.10%
5,Bulgaria,-58.50%
6,Belarus,-56.37%
7,Slovakia,-54.09%
8,Russian Federation,-52.02%
9,United Kingdom,-49.45%
